Imports

In [29]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import json
import math

Load the grid installation data and the gridtx dump

In [2]:
file = "data/grid-metering-point-dump-plus-zipcode-GMAPS-in-gridtx.csv"
meters_df = pd.read_csv(file)

file = "data/subscription_data.csv"
grid_installation_df = pd.read_csv(file)


dtype={'adjusts_id': 'object',
       'ediel_product_code': 'float64',
       'invoice_item_id': 'object',
       'parent_id': 'object',
       'meter_id': 'str'}
file = "data/gridtx-dump.csv"
gridtx_df = dd.read_csv(file, sep=";", dtype=dtype)


In [3]:
print(list(meters_df.columns))
print(list(grid_installation_df.columns))

['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'idgrid_data_table', 'created_atgrid_data_table', 'updated_atgrid_data_table', 'meter_number', 'ean', 'automatic_meter', 'street_address', 'grid_owner_id', 'metering_grid_area', 'zip_code_id', 'provider', 'settlement_method', 'market_balance_area', 'code', 'name', 'idzipcode_table', 'created_atzipcode_table', 'updated_atzipcode_table', 'country', 'full_address', 'lat', 'lng']
['Unnamed: 0.1', 'Unnamed: 0', 'address_id', 'installation_id', 'num_panels', 'ID', 'Provider', 'status', 'Order ID', 'Address', 'customer ID', 'Metering point', 'Created by', 'Start date', 'End date', 'subscription_id']


In [73]:
# Get unique subscription_id
subscription_ids = grid_installation_df['subscription_id'].unique()
print("Number of unique supscription ids: ", len(subscription_ids))

print("Number of rows in installations dataset: ", len(grid_installation_df))

# Drop nan values
grid_installation_df = grid_installation_df.dropna()
subscription_ids = grid_installation_df['subscription_id'].unique()
print("Number of unique supscription ids without nan-values: ", len(grid_installation_df))

print("Number of rows in installations dataset nan-values: ", len(subscription_ids))


Number of unique supscription ids:  767
Number of rows in installations dataset:  780
Number of unique supscription ids without nan-values:  780
Number of rows in installations dataset nan-values:  767


Create a dataframe with unique supscription_id/meter_id pairs

In [7]:
# Get unique column combination 
subscription_meter_mapping = gridtx_df[["subscription_id", "meter_id"]].drop_duplicates().compute()

In [8]:
print(subscription_meter_mapping.head())

                         subscription_id                              meter_id
0   1eca63f1-cb49-4452-a03e-525584fe8ae8  6402472c-bba0-412b-8a60-f02753f727ed
28                                   NaN  b460d11a-4a53-4815-bd53-eb6bfa93a864
29                                   NaN  cbb52ab9-d616-4c76-bb29-ae4e6a973589
31                                   NaN  fc915b4f-2af3-4796-a206-e1a362312783
33                                   NaN  cd51ab4f-7aca-4d0d-8dba-a378c34f6ad4


It is seen that some meters have no subscription-id

In [9]:
print("Value counts of meter_id")
print(subscription_meter_mapping["meter_id"].value_counts().head())
print()
print("Value counts of supscription_id")
print(subscription_meter_mapping["subscription_id"].value_counts().head())

Value counts of meter_id
cebee489-9010-4cd6-90fb-6cfcf4823b14    4
0668a9b4-2df1-441b-9cc6-0eedd1482e2e    4
a3913cb4-6bce-4079-b50e-7e113122bb0c    4
6402472c-bba0-412b-8a60-f02753f727ed    3
5ce74fe0-924f-4f4d-a360-012704c061ff    3
Name: meter_id, dtype: int64

Value counts of supscription_id
1eca63f1-cb49-4452-a03e-525584fe8ae8    1
801311c4-5269-4c1c-8692-d541576d0439    1
55c057f5-9989-4a18-a682-983c9c08a885    1
4dc8dfcd-01a0-4338-af9d-0d49442a539b    1
0c859392-5606-48ee-906d-c0c06244734e    1
Name: subscription_id, dtype: int64


We see that a meter can have multiple supscription ids. We suspect that this is caused by the supscription being updated

Figure out which meters have no supscription id 

In [10]:
supscription_meter_mapping_nona = subscription_meter_mapping.dropna()

print("Unique meters before removing NaN values: ", subscription_meter_mapping["meter_id"].unique().size)
print("Unique meters after moving NaN values: ", supscription_meter_mapping_nona["meter_id"].unique().size)
print()
print("The following meters have no subscripton id: ")
meters_with_no_supscription_id = list(set(subscription_meter_mapping["meter_id"].unique()) - set(supscription_meter_mapping_nona["meter_id"].unique()))

print(meters_with_no_supscription_id)

Unique meters before removing NaN values:  1969
Unique meters after moving NaN values:  1945

The following meters have no subscripton id: 
['eddb6025-bb82-42b5-a4a2-f65049b0e790', '5126bddb-d50a-4988-a709-5a2c7a1a3d94', '7b70c0b7-59d7-4b1d-a6fd-983d810aeaf6', '079e241f-e60e-4dd4-b8ce-48491777e93f', '6286b5cf-630c-4b7d-8838-7da2ec96c298', '7cb83999-bc7f-4b97-9f39-9545c572b5f6', 'df80d951-c902-46bc-9047-dc0175feb0e1', '2122de95-6440-406a-983b-3b0628dda438', 'd1c3dbac-8188-4195-b83c-0558532f92a1', 'd4481dcf-4789-4411-8a21-3d1e5c127c35', 'de7739ce-aead-4927-b3ae-ed5d9ce861c0', '2a3a5272-f8a5-4619-b345-7e084e8e11e8', 'daa17344-9b67-4a7e-869b-bae8f2edb45c', '343a8809-29c6-4b60-bd81-bc32e731a60f', '7f8926f2-da70-4bdd-9663-3b8916f2d9ea', '2a841ee0-456c-44ae-a33e-d969c41f0bc2', '49608306-15d1-4f51-b87d-5d990b17612c', '54cad99a-8ea5-4841-8f19-68c97d131d60', '517c7f3f-a82c-4cc0-a573-70c612af9c55', 'c7610a7e-9a06-467d-aacc-4c80da77d365', 'dee7ffd5-f9d8-41b8-a160-b2c7a0ae1ed8', '299a8b32-7484-4973

Create a dictionary to map meter ids to their size installation size (num_panels)

In [42]:
num_panels_dict = {}

i = 0
for mid in supscription_meter_mapping_nona["meter_id"]:
    sup_ids = supscription_meter_mapping_nona[supscription_meter_mapping_nona["meter_id"] == mid]["subscription_id"].unique()
    

    np_list = []
    for sid in sup_ids:
        # dataframe is empty
        if len(grid_installation_df[grid_installation_df["subscription_id"] == sid]):
            num_panels = grid_installation_df[grid_installation_df["subscription_id"] == sid]["num_panels"]
            np_list.append(num_panels.values[0])

    if len(np_list) == 0:
        # print(np.nasn)
        num_panels_dict[mid] = np.nan
    elif len(list(set(np_list))) != 1:
        # Print the meter id if there is a mismatch between number of panels across multiple supscriptions on the same meter (expected to be the same)
        print(mid)
    else:
        num_panels_dict[mid] = np_list[0]

# Save dict to json
with open('data/meters_num_panels_dict.json', 'w') as fp:
    json.dump(num_panels_dict, fp)

It is seen that there are no meters where the multiple supscription ids has a different amount of panels

Count amount of nan-values in the newly created dictionary

In [44]:
# count nan values in dict
nan_count = sum(1 for x in num_panels_dict.values() if math.isnan(x))
print("Number of meters with no installation size: ", nan_count)
print("Number of meters in total: ", len(num_panels_dict))

print("Percentage of of meters with installation size: ", (len(num_panels_dict)-nan_count)/len(num_panels_dict)*100)

Number of meters with no installation size:  1204
Number of meters in total:  1945
Percentage of of meters with installation size:  38.09768637532134


Load meters with 100% coverage and check how many nan values there are with regards to number of panels

In [46]:
# load year_coverage_consumption_100.csv
file = "data/year_coverage_consumption_100.csv"
year_coverage_consumption_df = pd.read_csv(file)


In [52]:
year_coverage_consumption_df["num_panels"] = year_coverage_consumption_df["meter_id"].map(num_panels_dict)

In [70]:
year_coverage_consumption_df["num_panels"].isna().value_counts()

False    98
True     39
Name: num_panels, dtype: int64

Check number of panels for the meter with the most observations

In [71]:
num_panels_dict["3ba47f27-33e8-4764-a390-d33ca37d625f"]

20.0